### Importando os pacotes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from scipy.stats import kendalltau
import warnings
warnings.filterwarnings("ignore")

### Abrindo o dataset

In [ ]:
dados = pd.read_csv('mushrooms.csv')
dados.head()

### Verificando os detalhes do dataset

In [ ]:
dados.info()

### Realizando uma análise descritiva

In [ ]:
# Verificando o balanceamento da variável "class"
dados['class'].value_counts()

In [ ]:
# Verificando a quantidade de valores diferentes de cada variável
for i in list(dados.columns):
    print(i, dados[i].nunique())

In [ ]:
# Verificando a distribuição dos valores de cada variável
for classe in list(dados.columns):
    print(classe,'\n', dados[classe].value_counts())

### Separando as variáveis explicativas da variável resposta e transformando as variáveis em dummies  

In [ ]:
X = dados.drop(columns=['class'])
y = dados[['class']]

In [ ]:
def create_encoder(data):
    dummy_encoder = OneHotEncoder(drop="first", sparse=False)
    dummy_encoder.fit(data)
    return dummy_encoder

def encode(data, enc, features):
    data_dummy = enc.transform(data)
    data_dummy = pd.DataFrame(data=data_dummy, columns=enc.get_feature_names_out())
    data_dummy = data_dummy[features]
    return data_dummy

In [ ]:
enc_X = create_encoder(X)
enc_y = create_encoder(y)

In [ ]:
features_X = enc_X.get_feature_names_out()
X = encode(X, enc_X, features_X)

In [ ]:
features_y = ['class_p']
y = encode(y, enc_y, features_y)

In [ ]:
X.head()

In [ ]:
y.head()

### Separando as variáveis de treino e teste e criando o modelo de Regressão Logística

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


print('Acurácia: %f' % accuracy_score(y_test, y_pred))
print('Precision: %f' % precision_score(y_test, y_pred))
print('Recall: %f' % recall_score(y_test, y_pred))

### Verificando quais as variáveis explicativas tem maior correlação com a variável resposta

In [ ]:
df_dummies = X.copy()

In [ ]:
df_dummies['class_p'] = y

In [ ]:
df_dummies.head()

In [ ]:
corr = df_dummies.corr(method="kendall")['class_p'].sort_values()
corr.head(10)

In [ ]:
corr.tail(10)

### Gerando o modelo de Regressão Linear para observar o P valor das variáveis

In [ ]:
model = OLS(y, add_constant(X)).fit()
tab = model.summary().tables[1].data

In [ ]:
model.summary().tables[1]

### Checando o P valor das variáveis de maior correlação

<img src='Imagem2.png' alt='Image' height='542' width='542'>

### Verificando a possibilidade de ocorrência de multicolinearidade das variáveis de interesse através do VIF 

In [ ]:
selected_one_hot_features = ['bruises_t', 'odor_n', 'gill-size_n', 'odor_f', 'ring-type_l', 'spore-print-color_w']
X_1 = X[selected_one_hot_features]
y_1 = y

In [ ]:
Xc = add_constant(X_1)
vif_dict = {}
for i, col in enumerate(Xc.columns):
    vif_dict[col] = vif(Xc.values, i)
    
ser_vif = pd.Series(vif_dict)
ser_vif.sort_values()

### Verificando a qualidade do modelo apenas com as variáveis de selecionadas

In [ ]:
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size = 0.25, random_state = 10)

model_1 = LogisticRegression()
model_1.fit(X_1_train, y_1_train)

y_1_pred = model_1.predict(X_1_test)

print('Acurácia: %f' % accuracy_score(y_1_test, y_1_pred))
print('Precision: %f' % precision_score(y_1_test, y_1_pred))
print('Recall: %f' % recall_score(y_1_test, y_1_pred))

#### Com este modelo conseguimos reduzir a necessidade de variáveis de 22 para 05. São elas: bruises, odor, ring-type, gill-size e spore-print-color.

### Testando modelo de Árvore de decisão com todas as variáveis e somente com as variáveis selecionadas

In [ ]:
# Com todas as variáveis
model_2 = DecisionTreeClassifier()
model_2.fit(X_train, y_train)

y_2_pred = model_2.predict(X_test)

print('Acurácia: %f' % accuracy_score(y_test, y_2_pred))
print('Precision: %f' % precision_score(y_test, y_2_pred))
print('Recall: %f' % recall_score(y_test, y_2_pred))

In [ ]:
# Com apenas as variáveis de interesse
model_3 = DecisionTreeClassifier()
model_3.fit(X_1_train, y_1_train)

y_2_pred = model_3.predict(X_1_test)

print('Acurácia: %f' % accuracy_score(y_1_test, y_2_pred))
print('Precision: %f' % precision_score(y_1_test, y_2_pred))
print('Recall: %f' % recall_score(y_1_test, y_2_pred))

### Inferencia (identificando a classe para novas amostras)

In [ ]:
selected_features = ['class', 'bruises', 'odor', 'ring-type', 'gill-size', 'spore-print-color']
dados_novo = dados.loc[:10, selected_features]
X_novo = dados_novo.drop(columns=['class'])
y_novo = dados_novo[['class']]  # Só para testar.

In [ ]:
X_novo

In [ ]:
enc_X_novo = create_encoder(X_novo)
enc_y_novo = create_encoder(y_novo)

In [ ]:
features_X_novo = enc_X_novo.get_feature_names_out()

In [ ]:
X_novo_aux = encode(X_novo, enc_X_novo, features_X_novo)
y_novo_aux = encode(y_novo, enc_y, features_y)

In [ ]:
X_novo_aux.head()

In [ ]:
X_novo = pd.DataFrame(columns=selected_one_hot_features)

In [ ]:
X_novo[features_X_novo] = X_novo_aux
X_novo = X_novo[selected_one_hot_features]
X_novo.fillna(0, inplace=True)

In [ ]:
X_novo.head()

In [ ]:
y_novo_pred = model_1.predict(X_novo)
y_novo_pred

In [ ]:
for real, pred in zip(y_novo.values, y_novo_pred):
    print(real, pred)